# **Machine learning for low-resource NLP**: Advancing AI for Linguistic Inclusion
Cross-lingual transfer learning and pseudo-labeling for multilingual named entity recognition

In [1]:
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display, Markdown

import torch
import torch.optim as optim
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [2]:
from model import BertBilstmCrf
from dataloader import create_dataloaders
from training import train_model, train_pseudo_labeling, evaluate_epoch
from config import BaseConfig, TrainConfig, FineTuneConfig, PseudoLabelingConfig

In [3]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    np.random.seed(seed)
    random.seed(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(BaseConfig.RANDOM_STATE)

In [4]:
def load_wikiann_datasets(language_codes, cutoff=None):

    language_data = {}
    for lang in language_codes:

        # Load raw data from hugging face
        lang_dataset = load_dataset("unimelb-nlp/wikiann", name=lang)

        # Get data from different splits and combine
        train_df = pd.DataFrame(lang_dataset["train"])
        val_df = pd.DataFrame(lang_dataset["validation"])
        test_df = pd.DataFrame(lang_dataset["test"])

        complete_df = pd.concat([train_df, val_df, test_df]).reset_index(drop=True)
        complete_df = complete_df.head(cutoff) if cutoff else complete_df

        # Split data into new train/val/test splits
        train, temp = train_test_split(
            complete_df, test_size=0.2, random_state=BaseConfig.RANDOM_STATE
        )
        val, test = train_test_split(
            temp, test_size=0.5, random_state=BaseConfig.RANDOM_STATE
        )

        language_data[lang] = {"train": train, "val": val, "test": test}

    return language_data


# Download and store data
low_resource_datasets = load_wikiann_datasets(BaseConfig.low_resource_langs)

In [5]:
def setup_optimizer(model, CONFIG):
    param_groups = []
    # Check model layers and add appropiate learning rates
    if hasattr(model, "bert"):
        param_groups.append(
            {"params": model.bert.parameters(), "lr": TrainConfig.BERT_LEARNING_RATE}
        )
    if hasattr(model, "lstm"):
        param_groups.append(
            {"params": model.lstm.parameters(), "lr": TrainConfig.LSTM_LEARNING_RATE}
        )
    if hasattr(model, "crf"):
        param_groups.append(
            {"params": model.crf.parameters(), "lr": TrainConfig.CRF_LEARNING_RATE}
        )
    optimizer = optim.Adam(param_groups, weight_decay=TrainConfig.WEIGHT_DECAY)

    return optimizer

### Baseline Experiment
Baseline BERT-BiLSTM-CRF model trained on multilingual NER data

In [ ]:
baseline_results = []

# Iterate through low-resource languages
for lang, lang_data in tqdm(low_resource_datasets.items(), ncols=80):

    train_loader, val_loader, test_loader = create_dataloaders(lang_data)

    # ------------------------------------------ TRAINING ------------------------------------------ #

    model = BertBilstmCrf(BaseConfig.NUM_TAGS).to(BaseConfig.DEVICE)
    optimizer = setup_optimizer(model, TrainConfig)
    best_model_state, train_f1, val_f1 = train_model(
        model, optimizer, train_loader, val_loader, TrainConfig
    )

    # ------------------------------------------ EVALUATION ------------------------------------------ #
    eval_model = BertBilstmCrf(BaseConfig.NUM_TAGS).to(BaseConfig.DEVICE)
    eval_model.load_state_dict(best_model_state, TrainConfig)
    test_loss, test_f1 = evaluate_epoch(eval_model, test_loader)

    # ------------------------------------------ RESULTS ------------------------------------------ #
    torch.save(best_model_state, f"models/{lang}_baseline.pth")

    baseline_results.append(
        {"language": lang, "train_f1": train_f1, "val_f1": val_f1, "test_f1": test_f1}
    )

# Save and display results
baseline = pd.DataFrame(baseline_results)
baseline.to_csv("results/baseline.csv", index=False)

markdown_table = baseline.to_markdown(index=False)
display(Markdown(markdown_table))

100%|█████████████████████████████████████████████| 6/6 [05:28<00:00, 54.80s/it]


| language   |   train_f1 |   val_f1 |   test_f1 |
|:-----------|-----------:|---------:|----------:|
| mg         |   0.993728 | 0.933673 |  0.960352 |
| fo         |   0.974224 | 0.897482 |  0.901099 |
| co         |   0.956204 | 0.852308 |  0.81323  |
| hsb        |   0.951443 | 0.923387 |  0.854578 |
| bh         |   0.981549 | 0.888689 |  0.80212  |
| cv         |   0.977741 | 0.892617 |  0.830443 |

### Cross-lingual Transfer Learning
A technique where a model trained on one language (usually with more labeled data) is adapted to perform well on another language, leveraging shared linguistic representations.

In [7]:
transfer_results = []

for augmentation_factor in tqdm(range(1, 24), ncols=80):

    high_resource_datasets = load_wikiann_datasets(
        BaseConfig.high_resource_langs, augmentation_factor * 240
    )

    # Iterate through low-resource and adjacent high-resource languages
    for (low_resource_lang, low_resource_data), (
        high_resource_lang,
        high_resource_data,
    ) in tqdm(
        zip(low_resource_datasets.items(), high_resource_datasets.items()),
        ncols=80,
        leave=False,
    ):

        high_train_loader, high_val_loader, _ = create_dataloaders(high_resource_data)
        low_train_loader, low_val_loader, low_test_loader = create_dataloaders(
            low_resource_data
        )

        # ------------------------------------------ PRE-TRAINING ------------------------------------------ #

        high_resource_model = BertBilstmCrf(BaseConfig.NUM_TAGS).to(BaseConfig.DEVICE)
        optimizer = setup_optimizer(high_resource_model, TrainConfig)

        high_resource_model_state, train_f1, val_f1 = train_model(
            high_resource_model,
            optimizer,
            high_train_loader,
            high_val_loader,
            TrainConfig,
        )

        # ------------------------------------------ FINE-TUNING ------------------------------------------ #

        model = BertBilstmCrf(BaseConfig.NUM_TAGS).to(BaseConfig.DEVICE)
        model.load_state_dict(high_resource_model_state)
        optimizer = setup_optimizer(model, FineTuneConfig)

        best_model_state, train_f1, val_f1 = train_model(
            model, optimizer, low_train_loader, low_val_loader, FineTuneConfig
        )

        # ------------------------------------------ EVALUATION ------------------------------------------ #

        eval_model = BertBilstmCrf(BaseConfig.NUM_TAGS).to(BaseConfig.DEVICE)
        eval_model.load_state_dict(best_model_state)
        test_loss, test_f1 = evaluate_epoch(eval_model, low_test_loader)

        # ------------------------------------------ RESULTS ------------------------------------------ #
        torch.save(
            best_model_state,
            f"models/{low_resource_lang}_{high_resource_lang}_transfer.pth",
        )

        baseline_performance = baseline.loc[
            baseline["language"] == low_resource_lang, "test_f1"
        ].item()
        improvement = (test_f1 - baseline_performance) / baseline_performance * 100

        transfer_results.append(
            {
                "high_resource_language": high_resource_lang,
                "low_resource_lang": low_resource_lang,
                "augmentation_factor": augmentation_factor,
                "train_f1": train_f1,
                "val_f1": val_f1,
                "test_f1": test_f1,
                "improvement": improvement,
            }
        )

        print(
            f"Aug: {augmentation_factor}  {low_resource_lang} Improvement over baseline: {improvement:.5f}"
        )

transfer_data = pd.DataFrame(transfer_results)
transfer_data.to_csv("results/test_transfer_learning.csv", index=False)

  0%|                                                    | 0/23 [00:00<?, ?it/s]

Aug: 1  mg Improvement over baseline: 0.00000


Aug: 1  fo Improvement over baseline: -0.34843


Aug: 1  co Improvement over baseline: 3.34928


Aug: 1  hsb Improvement over baseline: -2.94118


Aug: 1  bh Improvement over baseline: 8.59031


  4%|█▊                                       | 1/23 [06:41<2:27:19, 401.80s/it]

Aug: 1  cv Improvement over baseline: 3.24826


  4%|█▊                                       | 1/23 [07:28<2:44:24, 448.37s/it]


KeyboardInterrupt: 

### Iterative Pseudo Labeling
A semi-supervised learning approach where a model generates predictions on unlabeled data, selects confident predictions as pseudo-labels, and retrains iteratively to improve performance.

In [ ]:
iterative_pseudo_labeling_results = []

# Iterate through low-resource languages
high_resource_datasets = load_wikiann_datasets(BaseConfig.high_resource_langs, 10000)

for conf_quantile in [
    0.82,
    0.83,
    0.84,
    0.85,
    0.86,
    0.87,
    0.88,
    0.89,
    0.9,
    0.91,
    0.92,
    0.93,
    0.94,
    0.95,
    0.96,
    0.97,
    0.98,
    0.99,
]:
    PseudoLabelingConfig.CONFIDENCE_QUANTILE = conf_quantile
    for _ in range(3):

        # Iterate through low-resource and adjacent high-resource languages
        for (lang, low_resource_data), (_, high_resource_data) in tqdm(
            zip(low_resource_datasets.items(), high_resource_datasets.items()),
            ncols=80,
        ):

            train_loader, val_loader, test_loader = create_dataloaders(
                low_resource_data
            )
            unlabeled_data = high_resource_data["train"]

            # ------------------------------------------ TRAINING ------------------------------------------ #

            model = BertBilstmCrf(BaseConfig.NUM_TAGS).to(BaseConfig.DEVICE)
            model.load_state_dict(torch.load(f"models/{lang}_baseline.pth"))
            optimizer = setup_optimizer(model, PseudoLabelingConfig)
            best_model_state, train_f1, val_f1 = train_pseudo_labeling(
                model,
                optimizer,
                train_loader,
                val_loader,
                unlabeled_data,
                PseudoLabelingConfig,
            )

            # ------------------------------------------ EVALUATION ------------------------------------------ #

            eval_model = BertBilstmCrf(BaseConfig.NUM_TAGS).to(BaseConfig.DEVICE)
            eval_model.load_state_dict(best_model_state)
            test_loss, test_f1 = evaluate_epoch(eval_model, test_loader)

            # ------------------------------------------ RESULTS ------------------------------------------ #
            torch.save(best_model_state, f"models/{lang}_iterative_pseudo_labeling.pth")

            baseline_performance = baseline.loc[
                baseline["language"] == lang, "test_f1"
            ].item()
            improvement = (test_f1 - baseline_performance) / baseline_performance * 100

            iterative_pseudo_labeling_results.append(
                {
                    "language": lang,
                    "train_f1": train_f1,
                    "val_f1": val_f1,
                    "test_f1": test_f1,
                    "improvement": improvement,
                    "confidence_quantile": conf_quantile,
                }
            )

            print(f"Language: {lang}    Improvement over baseline: {improvement:.5f}")

# Save results
iterative_pseudo_labeling = pd.DataFrame(iterative_pseudo_labeling_results)
iterative_pseudo_labeling.to_csv(
    "results/second_full_iterative_pseudo_labeling.csv", index=False
)

0it [00:00, ?it/s]

Added 467 rows of data
Added 0 rows of data
Added 0 rows of data
Added 0 rows of data


1it [08:42, 522.31s/it]

Language: mg    Improvement over baseline: 0.22936


2it [12:53, 363.06s/it]

...

Language: hsb    Improvement over baseline: -2.31092
Added 30 rows of data
Added 0 rows of data
Added 0 rows of data
Added 0 rows of data
Added 0 rows of data


5it [23:06, 288.15s/it]

Language: bh    Improvement over baseline: 2.42291


6it [25:59, 259.95s/it]

Language: cv    Improvement over baseline: 2.55220


9:

class PseudoLabelingConfig(BaseConfig):
    EPOCHS                = 25
    PATIENCE              = 5
    BERT_LEARNING_RATE    = 0.00002
    LSTM_LEARNING_RATE    = 0.003
    CRF_LEARNING_RATE     = 0.00003

    CONFIDENCE_QUANTILE   = 0.95
    PSEUDO_DELAY          = 8
    ENTROPY_THRESHOLD     = 0.2

10:

class PseudoLabelingConfig(BaseConfig):
    EPOCHS                = 25
    PATIENCE              = 5
    BERT_LEARNING_RATE    = 0.00002
    LSTM_LEARNING_RATE    = 0.003
    CRF_LEARNING_RATE     = 0.00003

    CONFIDENCE_QUANTILE   = 0.90
    PSEUDO_DELAY          = 8
    ENTROPY_THRESHOLD     = 0.15

In [20]:
df = pd.read_csv("results/full_iterative_pseudo_labeling.csv")
df

,language,train_f1,val_f1,test_f1,improvement,confidence_quantile
0,mg,0.999543,0.943878,0.960352,0.000000,0.88
1,fo,0.992799,0.908273,0.912088,1.219512,0.88
2,co,0.998431,0.852308,0.883268,8.612440,0.88
3,hsb,0.963006,0.933468,0.838420,-1.890756,0.88
4,bh,0.995533,0.903052,0.826855,3.083700,0.88
...,...,...,...,...,...,...
175,fo,0.995790,0.917266,0.926217,2.787456,0.93
176,co,0.991126,0.870769,0.883268,8.612440,0.93
177,hsb,0.989406,0.951613,0.838420,-1.890756,0.93
178,bh,0.977077,0.881508,0.810954,1.101322,0.93


In [21]:
df[df["language"] == "hsb"]

,language,train_f1,val_f1,test_f1,improvement,confidence_quantile
3,hsb,0.963006,0.933468,0.838420,-1.890756,0.88
9,hsb,0.982419,0.939516,0.847397,-0.840336,0.88
15,hsb,0.982341,0.963710,0.856373,0.210084,0.88
21,hsb,0.989991,0.961694,0.825853,-3.361345,0.88
27,hsb,0.971733,0.933468,0.847397,-0.840336,0.88
33,hsb,0.985929,0.939516,0.867145,1.470588,0.89
39,hsb,0.940353,0.917339,0.813285,-4.831933,0.89
45,hsb,0.973022,0.959677,0.842011,-1.470588,0.89
51,hsb,0.996490,0.941532,0.859964,0.630252,0.89
57,hsb,0.989977,0.947581,0.838420,-1.890756,0.89
